# Load and Merge Data from JSON

In [15]:
import json
import pandas as pd

def load_and_merge_data(file_paths):
    data_list = []
    for file_path in file_paths:
        with open(file_path, 'r') as f:
            data = json.load(f)
            # Check if 'deposits' key exists before accessing it
            if 'deposits' in data:
                # Iterate through each deposit and add it to the list
                for deposit in data['deposits']:
                    # Ensure 'account' key exists in the deposit before accessing it
                    if 'account' in deposit:
                        data_list.append(deposit)
                    else:
                        print(f"Warning: 'account' key not found in a deposit within {file_path}")
            else:
                print(f"Warning: 'deposits' key not found in {file_path}")
    return pd.DataFrame(data_list)

file_paths = [
    "/content/compoundV2_transactions_ethereum_chunk_0.json",
    "/content/compoundV2_transactions_ethereum_chunk_1.json",
    "/content/compoundV2_transactions_ethereum_chunk_2.json"
]

df = load_and_merge_data(file_paths)

def preprocess_data(df):
    # Check if the 'account' column exists before processing it
    if 'account' in df.columns:
        # Convert timestamp to numeric (if needed) and then to datetime
        df['timestamp'] = pd.to_datetime(pd.to_numeric(df['timestamp'], errors='coerce'), unit='s')
        df['account_id'] = df['account'].apply(lambda x: x['id'])
        df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
        df['amountUSD'] = pd.to_numeric(df['amountUSD'], errors='coerce')
        df['asset_symbol'] = df['asset'].apply(lambda x: x['symbol'])
        df = df.drop(columns=['account', 'asset', 'id', 'hash'])
        df = df.dropna()
        return df
    else:
        print("Warning: 'account' column not found in DataFrame. Skipping preprocessing.")
        return df  # Return the original DataFrame without changes

df = preprocess_data(df)

In [ ]:
!ls -lh /content/compoundV2_transactions_ethereum_chunk_*.json

-rw-r--r-- 1 root root  20M May  5 09:39 /content/compoundV2_transactions_ethereum_chunk_2.json
-rw-r--r-- 1 root root  19M May  5 09:39 /content/compoundV2_transactions_ethereum_chunk_6.json
-rw-r--r-- 1 root root 4.7M May  5 09:38 /content/compoundV2_transactions_ethereum_chunk_93.json


In [16]:
import json
import pandas as pd

def load_and_merge_data(file_paths):
    records = []
    for fp in file_paths:
        data = json.load(open(fp))
        # loop through each possible key
        for tx_type in ('deposits','borrows','repays','withdrawals','liquidations'):
            if tx_type in data:
                for tx in data[tx_type]:
                    tx['tx_type'] = tx_type[:-1]  # e.g. 'deposit','borrow',...
                    records.append(tx)
    return pd.DataFrame(records)

In [17]:
def feature_engineering(df):
    # parse existing columns
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df['account_id'] = df['account']['id']
    df['amountUSD'] = pd.to_numeric(df['amountUSD'], errors='coerce')
    df = df.dropna(subset=['account_id','tx_type','amountUSD'])

    # group by wallet
    g = df.groupby('account_id')

    # deposit features (you already have these)
    feats = {
      'total_deposit_usd':        g.apply(lambda d: d.loc[d.tx_type=='deposit','amountUSD'].sum()),
      'deposit_count':            g.apply(lambda d: (d.tx_type=='deposit').sum()),
      # … your other deposit features …
    }

    # borrow features
    feats.update({
      'total_borrow_usd':         g.apply(lambda d: d.loc[d.tx_type=='borrow','amountUSD'].sum()),
      'borrow_count':             g.apply(lambda d: (d.tx_type=='borrow').sum()),
    })

    # repay features
    feats.update({
      'total_repay_usd':          g.apply(lambda d: d.loc[d.tx_type=='repay','amountUSD'].sum()),
      'repay_count':              g.apply(lambda d: (d.tx_type=='repay').sum()),
      # repay/borrow ratio
      'repay_borrow_ratio':       lambda df: df['total_repay_usd']/df['total_borrow_usd'].replace(0, pd.NA)
    })

    # withdrawal features
    feats.update({
      'total_withdraw_usd':       g.apply(lambda d: d.loc[d.tx_type=='withdrawal','amountUSD'].sum()),
      'withdraw_count':           g.apply(lambda d: (d.tx_type=='withdrawal').sum()),
    })

    # liquidation features
    feats.update({
      'liquidation_count':        g.apply(lambda d: (d.tx_type=='liquidation').sum()),
      'had_multiple_liquidations': lambda df: (df['liquidation_count'] > 1).astype(int)
    })

    # convert to DataFrame
    features_df = pd.DataFrame({k: v(df) if callable(v) else v for k,v in feats.items()})
    features_df = features_df.fillna(0)
    return features_df

In [ ]:
df.columns

Index(['amount', 'amountUSD', 'timestamp', 'account_id', 'asset_symbol'], dtype='object')

# Feature Engineering

In [18]:
def feature_engineering(df):
    account_grouped = df.groupby('account_id')

    total_deposit_amount_usd = account_grouped['amountUSD'].sum()
    max_deposit_amount_usd = account_grouped['amountUSD'].max()
    avg_deposit_amount_usd = account_grouped['amountUSD'].mean()
    transaction_count = account_grouped.size()
    unique_assets_interacted = account_grouped['asset_symbol'].nunique()
    time_since_first_transaction = (pd.to_datetime('now') - account_grouped['timestamp'].min()).dt.days
    time_since_last_transaction = (pd.to_datetime('now') - account_grouped['timestamp'].max()).dt.days
    amount_usd_std = account_grouped['amountUSD'].std().fillna(0)

    features_df = pd.DataFrame({
        'total_deposit_amount_usd': total_deposit_amount_usd,
        'max_deposit_amount_usd': max_deposit_amount_usd,
        'avg_deposit_amount_usd': avg_deposit_amount_usd,
        'transaction_count': transaction_count,
        'unique_assets_interacted': unique_assets_interacted,
        'time_since_first_transaction': time_since_first_transaction,
        'time_since_last_transaction': time_since_last_transaction,
        'amount_usd_std': amount_usd_std
    })

    features_df = features_df.fillna(0)
    return features_df

features_df = feature_engineering(df)
features_df.head()

,total_deposit_amount_usd,max_deposit_amount_usd,avg_deposit_amount_usd,transaction_count,unique_assets_interacted,time_since_first_transaction,time_since_last_transaction,amount_usd_std
account_id,,,,,,,,
0x000000aaee6a496aaf7b7452518781786313400f,770814.615710,24447.806979,10142.297575,76,3,2134,2104,5740.490392
0x00000a6bbb35ea3f6f8dbc94801738f490c30a8d,6.063779,5.058038,3.031889,2,1,2162,2159,2.865407
0x0006e4548aed4502ec8c844567840ce6ef1013f5,236378.577102,38343.145712,9455.143084,25,4,2181,2109,8744.263019
0x0007808d13026d3245351759abc4f90a7bdd50de,14.972313,9.981542,7.486156,2,1,2103,2103,3.529008
0x000be263b3ceb55d22ed9b40d7619f62c4862c80,84697.515172,9674.990000,4457.763956,19,3,2170,2098,2520.430642


# Credit Scoring

In [19]:
from sklearn.preprocessing import MinMaxScaler

def scoring_model(features_df):
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(features_df)
    scaled_df = pd.DataFrame(scaled_features, index=features_df.index, columns=features_df.columns)

    weights = {
        'total_deposit_amount_usd': 0.15,
        'max_deposit_amount_usd': 0.10,
        'avg_deposit_amount_usd': 0.10,
        'transaction_count': 0.20,
        'unique_assets_interacted': 0.10,
        'time_since_first_transaction': 0.15,
        'time_since_last_transaction': -0.10,
        'amount_usd_std': -0.10
    }

    scaled_df['raw_score'] = scaled_df.apply(
        lambda row: sum(row[col] * weights[col] for col in weights), axis=1
    )

    min_raw_score = scaled_df['raw_score'].min()
    max_raw_score = scaled_df['raw_score'].max()
    scaled_df['credit_score'] = ((scaled_df['raw_score'] - min_raw_score) /
                                 (max_raw_score - min_raw_score)) * 100

    return scaled_df[['credit_score']].sort_values(by='credit_score', ascending=False)

scored_wallets_df = scoring_model(features_df)
scored_wallets_df.head()

,credit_score
account_id,
0x37acfef331e6063c8507c2a69c97b4f78c770a5a,100.000000
0x42b9df65b219b3dd36ff330a4dd8f327a6ada990,55.952691
0x6168bb05ccb4aaded0169602bbb6e3cae12902c5,55.768644
0x4654f56a64301b9b582f843f97332d96ead11ff8,55.721840
0xdf0635793e91d4f8e7426dbd9ed08471186f428d,54.912935


# Save Top 1000 Wallets

In [20]:
def output_top_wallets_csv(scored_wallets_df, top_n=1000, filename="top_1000_wallets.csv"):
    top_wallets = scored_wallets_df.head(top_n)
    top_wallets.to_csv(filename)
    print(f"\nTop {top_n} wallets with credit scores saved to '{filename}'")

output_top_wallets_csv(scored_wallets_df)


Top 1000 wallets with credit scores saved to 'top_1000_wallets.csv'


In [ ]:
import pandas as pd

# Read the CSV file into a Pandas DataFrame
top_wallets_df = pd.read_csv('top_1000_wallets.csv')

# Display the DataFrame
display(top_wallets_df)

,account_id,credit_score
0,0xd4bb298f935cdfa7eb5e363d168f838bb65cdff4,100.000000
1,0x1344a36a1b56144c3bc62e7757377d288fde0369,84.565052
2,0xb7896fce748396ecfc240f5a0d3cc92ca42d7d84,83.421540
3,0x8888882f8f843896699869179fb6e4f7e3b58888,74.564349
4,0x78e96be52e38b3fc3445a2ed34a6e586ffab9631,61.184871
...,...,...
995,0xc8d92ff6726e6887322b2136a79ea19f72882dfb,20.290704
996,0x2b6a4f029de9f6a17c0413b393770cacf28f6c8c,20.288058
997,0x49b61b0b725be7ee97361318ea6711f5ccf55ed3,20.287965
998,0x69f329ee79c98a94765e4105adfd5a87458d919c,20.286991


In [21]:
def output_top_wallets_csv(scored_wallets_df, top_n=1000, filename="wallet_scores.csv"):
    # Select top wallets and rename columns
    top_wallets = scored_wallets_df.head(top_n)
    # Reset the index to make 'account_id' a column
    top_wallets = top_wallets.reset_index()
    # Rename the columns to expected names
    top_wallets = top_wallets.rename(columns={'account_id': 'wallet_address', 'credit_score': 'score'})
    # Only include 'wallet_address' and 'score' columns
    top_wallets = top_wallets[['wallet_address', 'score']] # Changed to wallet_address and score

    # Save to CSV
    top_wallets.to_csv(filename, index=False)  # index=False to exclude row numbers
    print(f"\nTop {top_n} wallets with credit scores saved to '{filename}'")

# Assuming 'scored_wallets_df' is your DataFrame with scores:
output_top_wallets_csv(scored_wallets_df)


Top 1000 wallets with credit scores saved to 'wallet_scores.csv'


In [22]:
# Code to create and save the CSV file)...

# Download the file
from google.colab import files
files.download('wallet_scores.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Analyze Top and Bottom Wallets

In [23]:
def analyze_wallets(df, scored_wallets_df, num_wallets=5):
    high_scoring_wallets = scored_wallets_df.nlargest(num_wallets, 'credit_score').index
    low_scoring_wallets = scored_wallets_df.nsmallest(num_wallets, 'credit_score').index

    print("\n--- High-Scoring Wallet Analysis ---")
    for wallet in high_scoring_wallets:
        wallet_data = df[df['account_id'] == wallet]
        print(f"\nWallet: {wallet}")
        print(f"  Total USD: {wallet_data['amountUSD'].sum()}")
        print(f"  Transactions: {len(wallet_data)}")
        print(f"  First Tx: {wallet_data['timestamp'].min()}")
        print(f"  Last Tx: {wallet_data['timestamp'].max()}")
        print(f"  Unique Assets: {wallet_data['asset_symbol'].nunique()}")

    print("\n--- Low-Scoring Wallet Analysis ---")
    for wallet in low_scoring_wallets:
        wallet_data = df[df['account_id'] == wallet]
        print(f"\nWallet: {wallet}")
        if not wallet_data.empty:
            print(f"  Total USD: {wallet_data['amountUSD'].sum()}")
            print(f"  Transactions: {len(wallet_data)}")
            print(f"  First Tx: {wallet_data['timestamp'].min()}")
            print(f"  Last Tx: {wallet_data['timestamp'].max()}")
            print(f"  Unique Assets: {wallet_data['asset_symbol'].nunique()}")
        else:
            print("  No transactions found.")

analyze_wallets(df, scored_wallets_df)


--- High-Scoring Wallet Analysis ---

Wallet: 0x37acfef331e6063c8507c2a69c97b4f78c770a5a
  Total USD: 39683482.16810073
  Transactions: 1084
  First Tx: 2019-06-29 21:38:52
  Last Tx: 2019-08-13 07:50:43
  Unique Assets: 2

Wallet: 0x42b9df65b219b3dd36ff330a4dd8f327a6ada990
  Total USD: 36.84354018182038
  Transactions: 95
  First Tx: 2019-05-13 09:45:33
  Last Tx: 2019-07-31 07:24:58
  Unique Assets: 7

Wallet: 0x6168bb05ccb4aaded0169602bbb6e3cae12902c5
  Total USD: 8801.385982399852
  Transactions: 118
  First Tx: 2019-05-25 03:41:33
  Last Tx: 2019-08-12 22:15:06
  Unique Assets: 7

Wallet: 0x4654f56a64301b9b582f843f97332d96ead11ff8
  Total USD: 128074.26006393187
  Transactions: 91
  First Tx: 2019-05-07 21:26:14
  Last Tx: 2019-08-07 22:17:21
  Unique Assets: 6

Wallet: 0xdf0635793e91d4f8e7426dbd9ed08471186f428d
  Total USD: 59.17333700025301
  Transactions: 37
  First Tx: 2019-05-14 14:54:02
  Last Tx: 2019-08-08 20:26:23
  Unique Assets: 7

--- Low-Scoring Wallet Analysis ---



In [24]:
import pandas as pd

def analyze_wallets(df, scored_wallets_df, num_wallets=5, output_file="wallet_analysis.md"):
    """
    Analyzes high-scoring and low-scoring wallets and generates a report.

    Args:
        df: The original DataFrame with transaction data.
        scored_wallets_df: DataFrame with wallet credit scores.
        num_wallets: Number of top and bottom wallets to analyze.
        output_file: Path to save the report.
    """

    high_scoring_wallets = scored_wallets_df.nlargest(num_wallets, 'credit_score').index
    low_scoring_wallets = scored_wallets_df.nsmallest(num_wallets, 'credit_score').index

    with open(output_file, 'w') as f:
        f.write("# Wallet Analysis\n\n")

        f.write("## High-Scoring Wallets\n\n")
        for wallet in high_scoring_wallets:
            wallet_data = df[df['account_id'] == wallet]
            f.write(f"**Wallet:** {wallet}\n")
            f.write(f"- Total USD: {wallet_data['amountUSD'].sum()}\n")
            f.write(f"- Transactions: {len(wallet_data)}\n")
            f.write(f"- First Tx: {wallet_data['timestamp'].min()}\n")
            f.write(f"- Last Tx: {wallet_data['timestamp'].max()}\n")
            f.write(f"- Unique Assets: {wallet_data['asset_symbol'].nunique()}\n\n")
            # Add observations and justifications for high score here
            # Example:
            f.write("- **Observations:** This wallet has a high total deposit amount and a long transaction history, indicating consistent usage.\n")
            f.write("- **Justification for High Score:** The high deposit amount and consistent activity likely contributed to the high credit score.\n\n")


        f.write("## Low-Scoring Wallets\n\n")
        for wallet in low_scoring_wallets:
            wallet_data = df[df['account_id'] == wallet]
            f.write(f"**Wallet:** {wallet}\n")
            if not wallet_data.empty:
                f.write(f"- Total USD: {wallet_data['amountUSD'].sum()}\n")
                f.write(f"- Transactions: {len(wallet_data)}\n")
                f.write(f"- First Tx: {wallet_data['timestamp'].min()}\n")
                f.write(f"- Last Tx: {wallet_data['timestamp'].max()}\n")
                f.write(f"- Unique Assets: {wallet_data['asset_symbol'].nunique()}\n\n")
                # Add observations and justifications for low score here
                # Example:
                f.write("- **Observations:** This wallet has a very low total deposit amount and few transactions.\n")
                f.write("- **Justification for Low Score:** The limited activity and low deposit amount might have resulted in a low credit score.\n\n")
            else:
                f.write("- No transactions found.\n\n")

    print(f"Wallet analysis saved to {output_file}")

# Call the function to generate the report
analyze_wallets(df, scored_wallets_df)

Wallet analysis saved to wallet_analysis.md


In [27]:
import pandas as pd
from google.colab import files
from IPython.display import display, HTML
!apt-get install -y wkhtmltopdf  # Install wkhtmltopdf
!pip install pdfkit  # Install pdfkit
import pdfkit

def analyze_wallets(df, scored_wallets_df, num_wallets=5, output_file="wallet_analysis.html"):
    """
    Analyzes high-scoring and low-scoring wallets and generates an HTML report.

    Args:
        df: The original DataFrame with transaction data.
        scored_wallets_df: DataFrame with wallet credit scores.
        num_wallets: Number of top and bottom wallets to analyze.
        output_file: Path to save the HTML report.
    """

    high_scoring_wallets = scored_wallets_df.nlargest(num_wallets, 'credit_score').index
    low_scoring_wallets = scored_wallets_df.nsmallest(num_wallets, 'credit_score').index

    html_content = """
    <h1>Wallet Analysis</h1>
    <h2>High-Scoring Wallets</h2>
    """

    for wallet in high_scoring_wallets:
        wallet_data = df[df['account_id'] == wallet]
        html_content += f"""
        <h3>Wallet: {wallet}</h3>
        <ul>
            <li>Total USD: {wallet_data['amountUSD'].sum()}</li>
            <li>Transactions: {len(wallet_data)}</li>
            <li>First Tx: {wallet_data['timestamp'].min()}</li>
            <li>Last Tx: {wallet_data['timestamp'].max()}</li>
            <li>Unique Assets: {wallet_data['asset_symbol'].nunique()}</li>
        </ul>
        <p><b>Observations:</b> This wallet has a high total deposit amount and a long transaction history, indicating consistent usage.</p>
        <p><b>Justification for High Score:</b> The high deposit amount and consistent activity likely contributed to the high credit score.</p>
        """

    html_content += """
    <h2>Low-Scoring Wallets</h2>
    """

    for wallet in low_scoring_wallets:
        wallet_data = df[df['account_id'] == wallet]
        html_content += f"""
        <h3>Wallet: {wallet}</h3>
        """
        if not wallet_data.empty:
            html_content += f"""
            <ul>
                <li>Total USD: {wallet_data['amountUSD'].sum()}</li>
                <li>Transactions: {len(wallet_data)}</li>
                <li>First Tx: {wallet_data['timestamp'].min()}</li>
                <li>Last Tx: {wallet_data['timestamp'].max()}</li>
                <li>Unique Assets: {wallet_data['asset_symbol'].nunique()}</li>
            </ul>
            <p><b>Observations:</b> This wallet has a very low total deposit amount and few transactions.</p>
            <p><b>Justification for Low Score:</b> The limited activity and low deposit amount might have resulted in a low credit score.</p>
            """
        else:
            html_content += "<p>No transactions found.</p>"

    # Save HTML report
    with open(output_file, 'w') as f:
        f.write(html_content)

    print(f"Wallet analysis saved to {output_file}")

    # Convert HTML to PDF
    pdfkit.from_file(output_file, 'wallet_analysis.pdf')

    # Download the PDF
    files.download('wallet_analysis.pdf')

# Assuming df and scored_wallets_df are already defined
analyze_wallets(df, scored_wallets_df)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon geoclue-2.0 glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas iio-sensor-proxy
  libavahi-core7 libavahi-glib1 libdaemon0 libevdev2 libgudev-1.0-0 libhyphen0
  libinput-bin libinput10 libjson-glib-1.0-0 libjson-glib-1.0-common
  libmbim-glib4 libmbim-proxy libmd4c0 libmm-glib0 libmtdev1 libnl-genl-3-200
  libnotify4 libnss-mdns libproxy1v5 libqmi-glib5 libqmi-proxy libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5positioning5 libqt5printsupport5
  libqt5qml5 libqt5qmlmodels5 libqt5quick5 libqt5sensors5 libqt5svg5
  libqt5webchannel5 libqt5webkit5 libqt5widgets5 libsoup2.4-1
  libsoup2.4-common libudev1 libwacom-bin libwacom-common libwacom9 libwoff1
  libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0 libxcb-util1
  libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>